In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 19.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import faiss

In [ ]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/1bfQYKxx9LiE7NuolcPqwTph8U9lcVaXs/export?gid=191559656&format=csv')
df.head()


,Ссылка,Статья,Аннотация на русском,Журнал,10 ключевых слов с определениением на русском
0,https://disk.yandex.ru/i/w73mubOIzoMuNQ,Piloting Industry 4.0 in SMEs with RAMI 4.0: a...,Эта работа представляет собой исследование вне...,"BAPTISTA L.F., BARATA J. Piloting Industry 4.0...",Enterprise architecture: корпоративная архитек...
1,https://disk.yandex.ru/i/scYY76KwMxt0_w,Research on management informatization constru...,Статья посвящена исследованию информатизации у...,ZHAO X. Research on management informatization...,Big data: Большие данные - Объёмы разнообразно...
2,https://disk.yandex.ru/i/5goB4WpoFxNk-g,The Role of Enterprise Architecture in the Dig...,Статья посвящена роли корпоративной архитектур...,MATHAROO S. The role of enterprise architectur...,Enterprise Architecture: Корпоративная архитек...
3,https://disk.yandex.ru/i/OEMSeCixr0icdA,Sustainability of Implementing Enterprise Arch...,Данная статья посвящена вопросам внедрения кор...,"HINDARTO D., INDRAJIT R. EKO, DAZK E. Sustaina...",Enterprise Architecture: Корпоративная архитек...
4,https://disk.yandex.ru/i/sCQ4g4CVYrqiFQ,Electric Utility Enterprise Architecture to Su...,Представлена архитектура предприятия для элект...,"PARRA I., RODRÍGUEZ A., ARROYO-FIGUEROA G. Ele...","Smart Grid: Умная энергосеть - Энергосистема, ..."


In [ ]:
df.tail()

,Ссылка,Статья,Аннотация на русском,Журнал,10 ключевых слов с определениением на русском
173,https://disk.yandex.ru/i/7irZU5qQBKV6lA,Сравнение архитектурных подходов к анализу инф...,Статья посвящена исследованию необходимости мо...,Инженерный вестник Дона,Архитектура предприятия (Enterprise architectu...
174,https://disk.yandex.ru/i/hbkEDrKSJRjSPg,Разработка архитектуры предприятия с использов...,Статья посвящена разработке архитектуры предпр...,Международный журнал открытых информационных т...,Архитектура предприятия (Enterprise architectu...
175,https://disk.yandex.ru/i/sPzn4okaWONUCg,Технология документирования процесса разработк...,В статье рассматривается проблема отсутствия э...,Всероссийский журнал,Разработка стратегии (Strategy development)Опр...
176,https://disk.yandex.ru/i/TTH143J15-Y1fA,Изучение подходов к построению архитектуры пре...,Статья посвящена изучению архитектуры предприя...,Менеджер,Архитектура предприятия (Enterprise architectu...
177,https://disk.yandex.ru/i/AFVTcE-bJnSp9g,Архитектура системы управления цифровыми двойн...,Статья посвящена разработке архитектуры систем...,Вестник РГРТУ,Цифровые двойники (Digital twins)Определение: ...


In [ ]:
documents = [' '.join(x) for x in zip (df['Статья'], df['Аннотация на русском'], df['10 ключевых слов с определениением на русском'])]
labels = list(df['Статья'])

In [ ]:
documents[0]

'Piloting Industry 4.0 in SMEs with RAMI 4.0: an enterprise architecture approach Эта работа представляет собой исследование внедрения подходов Industry 4.0 в малые и средние предприятия (МСП) с использованием референсной архитектурной модели Industrie 4.0 (RAMI 4.0). Основной задачей является разработка и внедрение комплексной архитектуры, облегчающей цифровую трансформацию бизнеса. Исследование включает разработку пилотного проекта, направленного на создание мобильного приложения, которое улучшает взаимодействие с клиентами и повышает прозрачность производственного процесса. Проект реализуется на примере небольшой технической фирмы, занимающейся нанесением покрытий на компоненты промышленной продукции.\nПредложенный подход иллюстрирует, как RAMI 4.0 может служить основой для построения цифровой экосистемы, обеспечивая согласованность технических, экономических и социальных аспектов трансформации. Полученные результаты показывают потенциал данного подхода для масштабируемого развертыв

In [ ]:
len(documents)

178

In [ ]:
model_name = "intfloat/multilingual-e5-large"                                             #Тяжелая нейронная сеть
#model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"                #Чуть полегче

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [ ]:
# Преобразование документов в векторы
def embed_documents(documents):
    inputs = tokenizer(documents, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state[:, 0, :]  # Используем [CLS] токен
    return embeddings.numpy()

# Индексирование документов
doc_embeddings = embed_documents(documents)

In [ ]:
pd.DataFrame(doc_embeddings).to_csv('aaa.csv',index=False)

In [ ]:
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

In [ ]:
D, I = index.search(doc_embeddings, k=178)

In [ ]:
pd.DataFrame(D).to_csv('bbb.csv',index=False)

In [ ]:
pd.DataFrame(I).to_csv('ccc.csv',index=False)

In [ ]:
def print_results(query, k=5):
  query_embedding = embed_documents([query])
  D, I = index.search(query_embedding, k=k)
  for el in range(k):
    print('На позиции {} стоит {} с расстоянием {}'.format(el, labels[I[0][el]], D[0][el]))


In [ ]:
query = 'Предиктивное обслуживание оборудования'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Предложение архитектуры решения для прогнозного обслуживания с использованием машинного обучения с расстоянием 336.82611083984375
На позиции 1 стоит Industrial Maintenance and the Digital Twin-An Architectural Assessment с расстоянием 360.1602783203125
На позиции 2 стоит Ретрофитинг цифровых двойников существующих систем с расстоянием 364.56378173828125
На позиции 3 стоит Data and Artificial Intelligence Strategy: A Conceptual Enterprise Big Data Cloud Architecture to Enable Market-Oriented Organisations с расстоянием 364.9326477050781
На позиции 4 стоит Моделирование архитектуры и инфраструктуры предприятия с расстоянием 371.01824951171875


In [ ]:
query = 'Алгоритм принятия решений'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Decision Support Systems and Business Strategy: A Conceptual Framework for Strategic Information Systems Planning с расстоянием 376.83038330078125
На позиции 1 стоит Разработка структурно-функционального подхода к детализации мотивационного расширения ArchiMate для реализации отраслевых архитектурных решений с расстоянием 392.8999938964844
На позиции 2 стоит Архитектура предприятия? Инжиниринг? Или информационно-управляющий комплекс? с расстоянием 403.9961853027344
На позиции 3 стоит Предложение архитектуры решения для прогнозного обслуживания с использованием машинного обучения с расстоянием 404.8060302734375
На позиции 4 стоит Построение эффективной архитектуры предприятия как необходимое условие адаптации к цифровой экономике с расстоянием 409.8486328125


In [ ]:
query = 'Снижение простоев оборудования'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Предложение архитектуры решения для прогнозного обслуживания с использованием машинного обучения с расстоянием 329.2572937011719
На позиции 1 стоит Моделирование архитектуры и инфраструктуры предприятия с расстоянием 357.758056640625
На позиции 2 стоит Архитектура предприятия? Инжиниринг? Или информационно-управляющий комплекс? с расстоянием 363.9715576171875
На позиции 3 стоит Industrial Maintenance and the Digital Twin-An Architectural Assessment с расстоянием 365.32794189453125
На позиции 4 стоит Сравнение архитектурных подходов к анализу информационной поддержки процессов предприятия с расстоянием 373.27288818359375


In [ ]:
query = 'Анализ данных'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Research on management informatization construction of electric power enterprise based on big data technology с расстоянием 406.05029296875
На позиции 1 стоит Roles and Capabilities of Enterprise Architecture in Big Data Analytics Technology Adoption and Implementation с расстоянием 423.50274658203125
На позиции 2 стоит Сетецентрическая модель и архитектура единого информационного пространства «Проектирование на заказ» сложных технических изделий с расстоянием 424.4337463378906
На позиции 3 стоит Анализ и развитие подходов к формированию стратегии реализации цифровых продуктов и сервисов с расстоянием 430.0346984863281
На позиции 4 стоит Анализ архитектуры предприятия с использованием визуальных средств моделирования с расстоянием 431.0558166503906


In [ ]:
query = 'Автоматизация'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Архитектура предприятия? Инжиниринг? Или информационно-управляющий комплекс? с расстоянием 392.8258056640625
На позиции 1 стоит Архитектура информационной системы компании с расстоянием 411.6463623046875
На позиции 2 стоит Investment Models for Enterprise Architecture (EA) and IT Architecture Projects within the Open Innovation Concept с расстоянием 415.4849853515625
На позиции 3 стоит Многоагентная система сетевого предприятия на основе цифровой платформы индустрии четвертого поколения с расстоянием 430.3037414550781
На позиции 4 стоит Enterprise Architecture as a Tool for Digital Transformation of Mining Enterprises in the Arctic с расстоянием 432.3798522949219


In [ ]:
query = 'Предиктивная аналитика'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Roles and Capabilities of Enterprise Architecture in Big Data Analytics Technology Adoption and Implementation с расстоянием 389.2027282714844
На позиции 1 стоит Предложение архитектуры решения для прогнозного обслуживания с использованием машинного обучения с расстоянием 392.32012939453125
На позиции 2 стоит Data and Artificial Intelligence Strategy: A Conceptual Enterprise Big Data Cloud Architecture to Enable Market-Oriented Organisations с расстоянием 401.3210144042969
На позиции 3 стоит Анализ архитектуры предприятия с использованием визуальных средств моделирования с расстоянием 405.5473937988281
На позиции 4 стоит Современные инструменты архитектуры предприятия ИТ-компании с расстоянием 408.8315734863281


In [ ]:
query = 'Цифровая трансформация'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Цифровая трансформация как фактор формирования архитектуры и ИТ-архитектуры предприятия с расстоянием 343.188232421875
На позиции 1 стоит Enterprise architectures for the digital transformation in small and medium-sized enterprises с расстоянием 355.4737548828125
На позиции 2 стоит Современные тенденции применения архитектурного подхода в цифровой трансформации предприятия с расстоянием 356.88629150390625
На позиции 3 стоит Важнейшие проблемы, возникающие в ходе цифровой трансформации предприятий, и подходы к их решению с расстоянием 364.0268859863281
На позиции 4 стоит Investment Models for Enterprise Architecture (EA) and IT Architecture Projects within the Open Innovation Concept с расстоянием 365.03778076171875


In [ ]:
query = 'Энергетическая гибкость'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит A component-based design approach for energy flexibility in cyber-physical manufacturing systems с расстоянием 334.9061584472656
На позиции 1 стоит Цифровые двойники в энергетике с расстоянием 339.0849914550781
На позиции 2 стоит The Role of Enterprise Architecture in the Digital Transformation of Energy Companies с расстоянием 357.74468994140625
На позиции 3 стоит Using Enterprise Architecture to Integrate Lean Manufacturing, Digitalization, and Sustainability: A Lean Enterprise Case Study in the Chemical Industry с расстоянием 358.16595458984375
На позиции 4 стоит Enterprise Architecture as a Tool for Digital Transformation of Mining Enterprises in the Arctic с расстоянием 358.732177734375


In [ ]:
query = 'Кибер-физические производственные системы'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит A component-based design approach for energy flexibility in cyber-physical manufacturing systems с расстоянием 324.853759765625
На позиции 1 стоит Подход к формированию виртуальной метаструктуры цифрового проектного двойника корпоративной информационной системы машиностроительного предприятия с расстоянием 339.93798828125
На позиции 2 стоит Cloud based cyber-physical systems: Network evaluation study с расстоянием 348.9039306640625
На позиции 3 стоит Многоагентная система сетевого предприятия на основе цифровой платформы индустрии четвертого поколения с расстоянием 350.1091613769531
На позиции 4 стоит Development of manufacturing execution systems in accordance with Industry 4.0 requirements: A review of standard- and ontology-based methodologies and tools с расстоянием 357.16571044921875


In [ ]:
query = 'Компонентно-ориентированное проектирование'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Разработка высокоуровневого дизайна программно-аналитического комплекса предприятия, обеспечивающего сквозное планирование с расстоянием 348.8984680175781
На позиции 1 стоит Сетецентрическая модель и архитектура единого информационного пространства «Проектирование на заказ» сложных технических изделий с расстоянием 350.607177734375
На позиции 2 стоит A component-based design approach for energy flexibility in cyber-physical manufacturing systems с расстоянием 353.6875
На позиции 3 стоит Data and Artificial Intelligence Strategy: A Conceptual Enterprise Big Data Cloud Architecture to Enable Market-Oriented Organisations с расстоянием 368.2220458984375
На позиции 4 стоит Enterprise Architecture for Industry 4.0: Proposed Solutions and Production Data for TOGAF-Based Modeling с расстоянием 372.31329345703125


In [ ]:
query = 'Промышленный интернет вещей'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Industrial Maintenance and the Digital Twin-An Architectural Assessment с расстоянием 326.9327697753906
На позиции 1 стоит Enterprise Architecture as a Tool for Digital Transformation of Mining Enterprises in the Arctic с расстоянием 329.8335876464844
На позиции 2 стоит Выравнивание IIoT и ISA-95 для совершенствования управления активами в перерабатывающих отраслях с расстоянием 330.4586181640625
На позиции 3 стоит Design choices for next-generation IIoT-connected MES/MOM: An empirical study on smart factories с расстоянием 336.62310791015625
На позиции 4 стоит Data and Artificial Intelligence Strategy: A Conceptual Enterprise Big Data Cloud Architecture to Enable Market-Oriented Organisations с расстоянием 337.5532531738281


In [ ]:
query = 'RAMI 4.0 (Reference Architectural Model Industrie 4.0)'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Reference Architectural Model Industrie 4.0 (RAMI 4.0)   с расстоянием 255.20474243164062
На позиции 1 стоит ГОСТ Р 59799–2021. Умное производство. Модель эталонной архитектуры Индустрии 4.0 (RAMI4.0) с расстоянием 266.92236328125
На позиции 2 стоит Piloting Industry 4.0 in SMEs with RAMI 4.0: an enterprise architecture approach с расстоянием 268.55615234375
На позиции 3 стоит Architecture Alignment and Interoperability с расстоянием 273.4259338378906
На позиции 4 стоит Alignment Report for Reference Architectural Model for Industrie 4.0/ Intelligent Manufacturing System Architecture с расстоянием 282.75689697265625


In [ ]:
query = 'Административная оболочка активов (Asset Administration Shell - AAS)'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Digitalization and enterprise architecture management: a perspective on benefits and challenges с расстоянием 328.41357421875
На позиции 1 стоит Enterprise architectures for the digital transformation in small and medium-sized enterprises с расстоянием 341.56243896484375
На позиции 2 стоит Многоагентная система сетевого предприятия на основе цифровой платформы индустрии четвертого поколения с расстоянием 343.5138854980469
На позиции 3 стоит Improving Agency Performance Using Information and Information Technology (Enterprise Architecture Assessment Framework v3.1) с расстоянием 343.6229248046875
На позиции 4 стоит Enterprise Architecture for Digital Transformation с расстоянием 344.69110107421875


In [ ]:
query = 'Машинное обучение'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Cooperative Robotics and Machine Learning for Smart Manufacturing: Platform Design and Trends Within the Context of Industrial Internet of Things с расстоянием 427.11114501953125
На позиции 1 стоит Предложение архитектуры решения для прогнозного обслуживания с использованием машинного обучения с расстоянием 430.3961181640625
На позиции 2 стоит Data and Artificial Intelligence Strategy: A Conceptual Enterprise Big Data Cloud Architecture to Enable Market-Oriented Organisations с расстоянием 432.71209716796875
На позиции 3 стоит Investment Models for Enterprise Architecture (EA) and IT Architecture Projects within the Open Innovation Concept с расстоянием 440.1752014160156
На позиции 4 стоит Типологизация и систематизация подходов к разработке стратегии компании: модели и методы из смежных наук с расстоянием 441.5631103515625


In [ ]:
query = 'Виртуальная инженерия (Virtual Engineering)'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Architecting the Firm – Coherency and Consistency in Managing the Enterprise с расстоянием 351.8538818359375
На позиции 1 стоит Подход к формированию виртуальной метаструктуры цифрового проектного двойника корпоративной информационной системы машиностроительного предприятия с расстоянием 353.86669921875
На позиции 2 стоит Enterprise Architecture for Digital Transformation с расстоянием 360.0633544921875
На позиции 3 стоит Industrial Maintenance and the Digital Twin-An Architectural Assessment с расстоянием 363.06005859375
На позиции 4 стоит Enterprise Architecture as a Tool for Digital Transformation of Mining Enterprises in the Arctic с расстоянием 365.8456115722656


In [ ]:
query = 'OPC UA (Open Platform Communications Unified Architecture)'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Цифровая промышленность. Унифицированная архитектура ОРС. Часть 1. Обзор и концепции с расстоянием 319.7960205078125
На позиции 1 стоит Enterprise Architecture: A Strategy to Achieve e-Government Dimension of Smart Village Using TOGAF ADM 9.2 с расстоянием 366.5107421875
На позиции 2 стоит Enterprise architectures for the digital transformation in small and medium-sized enterprises с расстоянием 370.663330078125
На позиции 3 стоит Enterprise Architecture Management for Business Strategy Modelling с расстоянием 371.16815185546875
На позиции 4 стоит Business Strategy Modelling based on Enterprise Architecture: A State of the Art Review с расстоянием 372.06494140625


In [ ]:
query = 'Техническое обслуживание и ремонт'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Моделирование архитектуры и инфраструктуры предприятия с расстоянием 383.14208984375
На позиции 1 стоит Industrial Maintenance and the Digital Twin-An Architectural Assessment с расстоянием 387.1136779785156
На позиции 2 стоит Анализ и развитие подходов к формированию стратегии реализации цифровых продуктов и сервисов с расстоянием 391.4576416015625
На позиции 3 стоит Архитектурные модели управляемые сервисными соглашениями с расстоянием 392.158935546875
На позиции 4 стоит Описание технической архитектуры системы управления предприятием связи с использованием тензорной методологии анализа систем с расстоянием 396.2536926269531


In [ ]:
query = 'Цифровой двойник'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Цифровые двойники в энергетике с расстоянием 334.8797607421875
На позиции 1 стоит Ретрофитинг цифровых двойников существующих систем с расстоянием 350.42840576171875
На позиции 2 стоит Архитектура системы управления цифровыми двойниками производств как основа интеграции различных моделей их представлений с расстоянием 352.38275146484375
На позиции 3 стоит Подход к формированию виртуальной метаструктуры цифрового проектного двойника корпоративной информационной системы машиностроительного предприятия с расстоянием 354.9791259765625
На позиции 4 стоит Digital Twins in Manufacturing: A RAMI 4.0 Compliant Concept с расстоянием 357.8473815917969


In [ ]:
query = 'Экономический эффект'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Цифровые двойники в энергетике с расстоянием 394.990478515625
На позиции 1 стоит Institutional Perspectives on the Process of Enterprise Architecture Adoption с расстоянием 402.10546875
На позиции 2 стоит Enterprise Architecture for e-Strategy Standardization and Management: Lessons Learnt from Greece с расстоянием 416.23699951171875
На позиции 3 стоит Digitalization and enterprise architecture management: a perspective on benefits and challenges с расстоянием 424.04034423828125
На позиции 4 стоит Improving Agency Performance Using Information and Information Technology (Enterprise Architecture Assessment Framework v3.1) с расстоянием 425.6169128417969


In [ ]:
query = 'Высокотехнологичные предприятия'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Enterprise Architecture for Accounting and Finance Transformation: Using Strategy Maps to Develop High-Performance Finance с расстоянием 366.921875
На позиции 1 стоит Enterprise Architecture as a Tool for Digital Transformation of Mining Enterprises in the Arctic с расстоянием 367.81036376953125
На позиции 2 стоит Пути улучшения использования ИТ-архитектуры предприятия с расстоянием 371.8662109375
На позиции 3 стоит Архитектура предприятия? Инжиниринг? Или информационно-управляющий комплекс? с расстоянием 376.3741455078125
На позиции 4 стоит Research on management informatization construction of electric power enterprise based on big data technology с расстоянием 377.0616149902344


In [ ]:
query = 'Управление архитектурой предприятия'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Enterprise Architecture Management for Business Strategy Modelling с расстоянием 266.8367919921875
На позиции 1 стоит Enterprise Architecture Design Strategies for UGK Using TOGAF ADM с расстоянием 270.7543640136719
На позиции 2 стоит Enterprise Architecture Handbook с расстоянием 281.791259765625
На позиции 3 стоит Концепция системы поддержки архитектуры предприятия с расстоянием 282.42620849609375
На позиции 4 стоит Современные инструменты архитектуры предприятия ИТ-компании с расстоянием 284.8961181640625


In [ ]:
query = 'Предприятие топливно-энергетического сектора'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Enterprise Architecture as a Tool for Digital Transformation of Mining Enterprises in the Arctic с расстоянием 365.94525146484375
На позиции 1 стоит Using Enterprise Architecture to Integrate Lean Manufacturing, Digitalization, and Sustainability: A Lean Enterprise Case Study in the Chemical Industry с расстоянием 371.72711181640625
На позиции 2 стоит Electric Utility Enterprise Architecture to Support the Smart Grid с расстоянием 372.59954833984375
На позиции 3 стоит Enterprise Business Architecture as a Tool for Sustainable Development in an Enterprise - case study с расстоянием 373.09454345703125
На позиции 4 стоит Пути улучшения использования ИТ-архитектуры предприятия с расстоянием 375.4661560058594


In [ ]:
query = 'Газпром'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Enterprise Architecture as a Tool for Digital Transformation of Mining Enterprises in the Arctic с расстоянием 429.8968811035156
На позиции 1 стоит Анализ и развитие подходов к формированию стратегии реализации цифровых продуктов и сервисов с расстоянием 434.376953125
На позиции 2 стоит Using Enterprise Architecture to Integrate Lean Manufacturing, Digitalization, and Sustainability: A Lean Enterprise Case Study in the Chemical Industry с расстоянием 443.5025634765625
На позиции 3 стоит Enterprise Business Architecture as a Tool for Sustainable Development in an Enterprise - case study с расстоянием 446.1255187988281
На позиции 4 стоит Разработка архитектуры предприятия с использованием TOGAF ADM (на примере PT Dirgamitra Pasifik) с расстоянием 447.31964111328125


In [ ]:
query = 'Транспорт газа'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Enterprise Architecture as a Tool for Digital Transformation of Mining Enterprises in the Arctic с расстоянием 418.7235107421875
На позиции 1 стоит Using Enterprise Architecture to Integrate Lean Manufacturing, Digitalization, and Sustainability: A Lean Enterprise Case Study in the Chemical Industry с расстоянием 429.42791748046875
На позиции 2 стоит Сетецентрическая модель и архитектура единого информационного пространства «Проектирование на заказ» сложных технических изделий с расстоянием 432.43463134765625
На позиции 3 стоит Архитектура предприятия? Инжиниринг? Или информационно-управляющий комплекс? с расстоянием 434.4161071777344
На позиции 4 стоит Современные тенденции применения архитектурного подхода в цифровой трансформации предприятия с расстоянием 434.77001953125


In [ ]:
query = 'Технологии управления данными'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Research on management informatization construction of electric power enterprise based on big data technology с расстоянием 370.6592102050781
На позиции 1 стоит Архитектура предприятия? Инжиниринг? Или информационно-управляющий комплекс? с расстоянием 395.7164001464844
На позиции 2 стоит Пути улучшения использования ИТ-архитектуры предприятия с расстоянием 396.37445068359375
На позиции 3 стоит Decision Support Systems and Business Strategy: A Conceptual Framework for Strategic Information Systems Planning с расстоянием 402.53448486328125
На позиции 4 стоит Data and Artificial Intelligence Strategy: A Conceptual Enterprise Big Data Cloud Architecture to Enable Market-Oriented Organisations с расстоянием 409.416748046875


In [ ]:
query = 'Искусственные нейронные сети'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Сетецентрическая модель и архитектура единого информационного пространства «Проектирование на заказ» сложных технических изделий с расстоянием 396.43817138671875
На позиции 1 стоит Cloud based cyber-physical systems: Network evaluation study с расстоянием 400.99468994140625
На позиции 2 стоит A component-based design approach for energy flexibility in cyber-physical manufacturing systems с расстоянием 403.8536376953125
На позиции 3 стоит Architecting the Firm – Coherency and Consistency in Managing the Enterprise с расстоянием 406.416015625
На позиции 4 стоит Decision Support Systems and Business Strategy: A Conceptual Framework for Strategic Information Systems Planning с расстоянием 409.2617492675781


In [ ]:
query = 'Преподавание математики в школе'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит ENTERPRISE ARCHITECTURE DESIGN VISION ARCHITECTURE AND BUSINESS ARCHITECTURE STAGE USING TOGAF ADM AT SMA ABC с расстоянием 395.53277587890625
На позиции 1 стоит Enterprise Architecture as a Tool for Digital Transformation of Mining Enterprises in the Arctic с расстоянием 398.8237609863281
На позиции 2 стоит Анализ и развитие подходов к формированию стратегии реализации цифровых продуктов и сервисов с расстоянием 400.983154296875
На позиции 3 стоит Архитектура предприятия? Инжиниринг? Или информационно-управляющий комплекс? с расстоянием 404.331298828125
На позиции 4 стоит Ретрофитинг цифровых двойников существующих систем с расстоянием 412.7614440917969


In [ ]:
query = 'ИТ-отрасль активно развивается\n С ней связывается все больше аспектов человеческой и общественной жизни. Менеджмент на предприятиях — не исключение. Бизнес использует большое количество решений при выполнении поставленных задач. Информационные системы и технологии (ИТ) в сфере управления организацией — это методы, которые позволяют эффективно производить планирование, обмениваться данными, контролировать поставки, а также совершать другие действия, направленные на оптимизацию рабочих процессов и максимизации прибыли. Они выполняются на базе компьютеров или иной техники.'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Investment Models for Enterprise Architecture (EA) and IT Architecture Projects within the Open Innovation Concept с расстоянием 243.420654296875
На позиции 1 стоит Пути улучшения использования ИТ-архитектуры предприятия с расстоянием 254.35882568359375
На позиции 2 стоит От стратегии предприятия, через его архитектуру, к IT-стратегии с расстоянием 254.64300537109375
На позиции 3 стоит Electronic Enterprise: Strategy and Architecture с расстоянием 263.44775390625
На позиции 4 стоит Enterprise Architecture as Information Technology Strategy с расстоянием 266.9862365722656


In [ ]:
query = 'Инновационный бизнес даже более других видов коммерческой деятельности нуждается в стратегическом планировании. Стратегическое планирование обеспечивает способность инновационного бизнеса быть конкурентоспособным через концентрацию инновационных ресурсов, существенное улучшение результатов инновационной деятельности и раскрытие тех инновационных возможностей, которые позволили бы получить инвестиционные ресурсы. Для тех инвесторов и кредиторов, которые рассматривают вопросы финансирования инновационного бизнеса, стратегия развития инновационного предприятия позволяет выделить предприятие из среды конкурентов и повысить шансы получения финансирования.'
k=5 # Выводим 5 наиболее похожих тем
print_results(query, k)

На позиции 0 стоит Систематизация принципов и методология стратегического планирования в условиях цифровой трансформации с расстоянием 238.10975646972656
На позиции 1 стоит Стратегическое управление цифровой трансформацией предприятия: концепция и инструменты с расстоянием 268.57489013671875
На позиции 2 стоит Архитектурный подход к разработке стратегии цифровой трансформации предприятия с расстоянием 275.68994140625
На позиции 3 стоит AI-Driven Enterprise Architecture and Strategic Alignment с расстоянием 276.37164306640625
На позиции 4 стоит Process for Leveraging Enterprise Architecture in Information Systems Strategic Planning: A Case of Developing a Strategy and Master Plan for a National Integrated Health Laboratory Information Management System in Uganda с расстоянием 281.1620178222656
